In [1]:
import pandas as pd
import numpy as np
import random

from deap import base
from deap import creator
from deap import tools
from deap import algorithms

In [2]:
pessoas = [
    {"nome": "João", "origem": "CWB", "destino": "GRU"},
    {"nome": "Maria", "origem": "GIG", "destino": "GRU"},
    {"nome": "Pedro", "origem": "POA", "destino": "GRU"},
    {"nome": "Ana", "origem": "FLN", "destino": "GRU"},
    {"nome": "José", "origem": "CNF", "destino": "GRU"},
    {"nome": "Paulo", "origem": "GYN", "destino": "GRU"},
]

df_pessoas = pd.DataFrame(pessoas)

In [3]:
with open("voos.txt") as f:
    voos_data = [line.rstrip() for line in f]

keys = ["origem", "destino", "horario_partida", "horario_chegada", "valor"]
voos_list = [dict(zip(keys, voo.split(","))) for voo in voos_data]

df_voos = pd.DataFrame(voos_list)
df_voos["valor"] = df_voos["valor"].astype(float)
df_voos.reset_index(inplace=True)

# calculando o tempo de voo em minutos

df_voos["horario_partida_minutos"] = df_voos["horario_partida"].str.split(":").str[
    0
].astype(int) * 60 + df_voos["horario_partida"].str.split(":").str[1].astype(int)
df_voos["horario_chegada_minutos"] = df_voos["horario_chegada"].str.split(":").str[
    0
].astype(int) * 60 + df_voos["horario_chegada"].str.split(":").str[1].astype(int)
df_voos["tempo_voo"] = (
    df_voos["horario_chegada_minutos"] - df_voos["horario_partida_minutos"]
)

In [4]:
def imprime_agenda(individuo):
    print("Agenda:") 
    return df_voos[df_voos["index"].astype(int).isin(individuo)]

In [5]:
def avaliacao(individuo):
    df_temp = df_voos[df_voos["index"].isin(individuo)].copy()

    preco_total = df_temp["valor"].sum()
    ultima_chegada = df_temp["horario_chegada_minutos"].max()

    primeira_partida = df_temp["horario_partida_minutos"].min()

    df_temp['tempo_espera'] = (ultima_chegada - df_temp["horario_chegada_minutos"]) + (
        df_temp["horario_partida_minutos"] - primeira_partida
    )

    if ultima_chegada > primeira_partida:
        preco_total += 100

    total_espera = df_temp["tempo_espera"].sum()

    return (preco_total + total_espera,)

In [6]:
def inicializacao_par_voos(origem, destino):
    ida_valida = np.random.choice(
        df_voos.loc[
            (df_voos["origem"] == origem) & (df_voos["destino"] == destino)
        ].index
    ).tolist()
    volta_valida = np.random.choice(
        df_voos.loc[
            (df_voos["origem"] == destino) & (df_voos["destino"] == origem)
        ].index
    ).tolist()
    return ida_valida, volta_valida

In [7]:
def cxHalfAndHalf(ind1, ind2):

    size = min(len(ind1), len(ind2))
    cxpoint = size // 2 

    ind1[cxpoint:], ind2[cxpoint:] = ind2[cxpoint:], ind1[cxpoint:]

    return ind1, ind2

In [8]:
def mut_voos(individual):
    index = 2 * random.randint(0, len(individual) // 2 - 1)
    
    voo_ida = df_voos.iloc[individual[index]]
    origem, destino = voo_ida['origem'], voo_ida['destino']

    nova_ida = df_voos[(df_voos['origem'] == origem) & (df_voos['destino'] == destino)].sample(n=1).index.item()
    nova_volta = df_voos[(df_voos['origem'] == destino) & (df_voos['destino'] == origem)].sample(n=1).index.item()
        
    individual[index] = nova_ida
    individual[index + 1] = nova_volta
    
    return individual,

In [9]:
def create_individual(df_pessoas):
    individual = []
    for _, origem, destino in df_pessoas.itertuples(index=False):
        individual.extend(inicializacao_par_voos(origem, destino))
    return creator.Individual(individual)

In [10]:
# setup deap

# o individuo é uma lista de voos, cada voo é um dicionario
# a cada 2 voos, temos 1 pessoa ida e volta

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# individuo
toolbox.register("individual", create_individual, df_pessoas)
# populacao
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
# avaliacao
toolbox.register("evaluate", avaliacao)
# crossover
toolbox.register("mate", tools.cxTwoPoint)
# mutacao
toolbox.register("mutate", mut_voos)
# selecao
toolbox.register("select", tools.selTournament, tournsize=3)

In [11]:
if __name__ == "__main__":
    populacao = toolbox.population(n=300)

    CXPB, MUTPB, NGEN = 0.5 , 0.1, 200
    estatisticas = tools.Statistics(key=lambda individuo: individuo.fitness.values)
    estatisticas.register("min", np.min)

    populacao, info = algorithms.eaSimple(
        population=populacao,
        toolbox=toolbox,
        cxpb=CXPB,
        mutpb=MUTPB,
        ngen=NGEN,
        stats=estatisticas,
        verbose=False,
    )

    populacao, info
    melhores = tools.selBest(populacao, k=1)

In [12]:
info

[{'gen': 0, 'nevals': 300, 'min': 7419.0},
 {'gen': 1, 'nevals': 179, 'min': 7256.0},
 {'gen': 2, 'nevals': 180, 'min': 7256.0},
 {'gen': 3, 'nevals': 143, 'min': 7070.0},
 {'gen': 4, 'nevals': 146, 'min': 6698.0},
 {'gen': 5, 'nevals': 173, 'min': 5666.0},
 {'gen': 6, 'nevals': 160, 'min': 5095.0},
 {'gen': 7, 'nevals': 157, 'min': 5095.0},
 {'gen': 8, 'nevals': 154, 'min': 5154.0},
 {'gen': 9, 'nevals': 153, 'min': 3991.0},
 {'gen': 10, 'nevals': 169, 'min': 3991.0},
 {'gen': 11, 'nevals': 174, 'min': 3991.0},
 {'gen': 12, 'nevals': 154, 'min': 3991.0},
 {'gen': 13, 'nevals': 148, 'min': 3991.0},
 {'gen': 14, 'nevals': 160, 'min': 3870.0},
 {'gen': 15, 'nevals': 185, 'min': 3712.0},
 {'gen': 16, 'nevals': 171, 'min': 3712.0},
 {'gen': 17, 'nevals': 183, 'min': 3712.0},
 {'gen': 18, 'nevals': 153, 'min': 3591.0},
 {'gen': 19, 'nevals': 169, 'min': 3591.0},
 {'gen': 20, 'nevals': 161, 'min': 3543.0},
 {'gen': 21, 'nevals': 155, 'min': 3543.0},
 {'gen': 22, 'nevals': 158, 'min': 3543.0}

In [13]:
melhores[0]

[67, 66, 87, 86, 109, 106, 47, 46, 29, 26, 9, 6]

In [14]:
imprime_agenda(melhores[0])

Agenda:


,index,origem,destino,horario_partida,horario_chegada,valor,horario_partida_minutos,horario_chegada_minutos,tempo_voo
6,6,GRU,GYN,11:07,13:24,171.0,667,804,137
9,9,GYN,GRU,12:18,14:56,172.0,738,896,158
26,26,GRU,CNF,10:33,13:11,132.0,633,791,158
29,29,CNF,GRU,12:44,14:17,134.0,764,857,93
46,46,GRU,FLN,11:08,14:38,262.0,668,878,210
47,47,FLN,GRU,11:28,14:40,248.0,688,880,192
66,66,GRU,CWB,10:33,12:03,74.0,633,723,90
67,67,CWB,GRU,11:16,13:29,83.0,676,809,133
86,86,GRU,GIG,10:51,14:16,256.0,651,856,205
87,87,GIG,GRU,10:30,14:57,290.0,630,897,267
